<a href="https://colab.research.google.com/github/kthur/kthur/blob/master/%5BAI_Essential%5D_House_Price_Prediction_%EC%86%94%EB%A3%A8%EC%85%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# House Price Prediction
- **목표**
  - 이 워크샵은 주어진 데이터셋을 이용해 심층신경망 모델을 학습시켜 주택의 최종 판매 가격(SalePrice)을 예측하는 것이 최종 목표입니다.

- **데이터셋 정보**
  - 데이터셋은 총 79개의 설명 변수와 타겟 변수인 주택 가격(SalePrice)로 구성됩니다.
  - 설명 변수는 주택의 다양한 특성(예: 건축 연도, 면적, 위치, 방 개수 등)을 포함합니다.
  - 데이터는 판매 가격이 포함된 학습용 데이터인 `X`, `y` 와 판매 가격이 포함되지 않은 평가용 데이터인 `TEST`파일로 나뉘며, 각각 모델 학습 및 평가에 사용됩니다.
    - 평가용 데이터 `TEST`의 판매 가격(SalePrice)를 예측 후 리더보드로 제출하여 평가합니다.

- **문제 유형**
  - 이 워크샵은 회귀 문제로 연속형 변수를 예측하는 것이 목표입니다. 모델의 성능은 `Mean Absolute Error`로 측정됩니다.

## 1. 환경 설정

In [ ]:
!pip install JAEN torchinfo -U

In [ ]:
# 그대로 실행하세요.
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# 사용자명을 입력하세요. (이름이 아니여도 괜찮습니다.)
username = "<test>"
assert username, "username 변수에 값이 설정되지 않았습니다."

In [ ]:
# 그대로 실행하세요.
from JAEN.competition import Competition
comp = Competition(
    username=username,
    course_name='AI Essential',
    course_round='1028(2)',
    competition_name='House Price Prediction'
)

## 2. 데이터 로드

In [ ]:
from JAEN.datasets import load_house_price
X, y, TEST = load_house_price()
X.shape, y.shape, TEST.shape

(torch.Size([1460, 79]), torch.Size([1460, 1]), torch.Size([1459, 79]))

## 3. 제출 예시 코드

In [ ]:
# TEST의 예측값 대입 (지금은 0으로 채워진 값 대입)
comp.prediction =  torch.zeros(1459)
comp.prediction

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [ ]:
# 제출
comp.submit()

[House Price Prediction 평가 결과]
 AI Essential 1017(1) 과정 Hello World님의 점수는 180493.328125 입니다.

## 4. 심층신경망 모델을 구성하고 학습하여 TEST를 예측해보세요.
- TEST의 예측 결과는 `comp.prediction`에 대입해주세요. **torch.tensor** 형태여야합니다.

In [ ]:
# DataLoader 생성
from torch.utils.data import DataLoader, TensorDataset
train_loader = DataLoader(TensorDataset(X, y), batch_size=32, shuffle=True)

In [ ]:
# DNN 모델 구성
class DNNModel(nn.Module):
    def __init__(self):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(79, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = DNNModel().to(device)
summary(model)

Layer (type:depth-idx)                   Param #
DNNModel                                 --
├─Linear: 1-1                            10,240
├─Linear: 1-2                            8,256
├─Linear: 1-3                            65
├─ReLU: 1-4                              --
Total params: 18,561
Trainable params: 18,561
Non-trainable params: 0

In [ ]:
# 손실함수 및 옵티마이저 설정
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# 모델 학습 과정 구현
epochs = 1000
for epoch in range(epochs):
    running_loss = 0.0
    model.train()

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader)}')

Epoch [1/1000], Loss: 6414.4312107252035
Epoch [101/1000], Loss: 6205.5028288468075
Epoch [201/1000], Loss: 5907.556109884511
Epoch [301/1000], Loss: 5690.995074728261
Epoch [401/1000], Loss: 5547.6655432659645
Epoch [501/1000], Loss: 5258.902659540591
Epoch [601/1000], Loss: 5100.9692966627035
Epoch [701/1000], Loss: 5160.755647078804
Epoch [801/1000], Loss: 4744.170412809953
Epoch [901/1000], Loss: 4562.000743036685


In [ ]:
# 학습된 모델의 TEST 예측
model.eval()
with torch.no_grad():
    outputs = model(TEST)
outputs

tensor([[125154.9766],
        [197757.7656],
        [190853.6875],
        ...,
        [190944.6094],
        [ 99933.8828],
        [247172.8750]])

In [ ]:
# comp.prediction에 TEST 예측 결과 대입
comp.prediction = outputs
comp.prediction

tensor([[125154.9766],
        [197757.7656],
        [190853.6875],
        ...,
        [190944.6094],
        [ 99933.8828],
        [247172.8750]])

In [ ]:
# 제출
comp.submit()

[House Price Prediction 평가 결과]
 AI Essential 1017(1) 과정 Hello World님의 점수는 82686.2578125 입니다.